# PyTorchを使ってLSTMで文章分類を実装してみた

https://qiita.com/m__k/items/841950a57a0d7ff05506

In [1]:
import os
from glob import glob
import pandas as pd
import numpy as np
import linecache
from IPython.display import Image

In [2]:
# カテゴリを配列で取得
categories = [name for name in os.listdir("text") if os.path.isdir("text/" + name)]
print(categories)

['dokujo-tsushin', 'it-life-hack', 'kaden-channel', 'livedoor-homme', 'movie-enter', 'peachy', 'smax', 'sports-watch', 'topic-news']


In [3]:
datasets = pd.DataFrame(columns=["title", "category"])
print(f"type: ${type(datasets)}")
print(f"shape: ${datasets.shape}")
datasets

type: $<class 'pandas.core.frame.DataFrame'>
shape: $(0, 2)


,title,category


In [4]:
for cat in categories:
    path = "text/" + cat + "/*.txt"
    files = glob(path)
    for text_name in files:
        title = linecache.getline(text_name, 3)
        s = pd.Series([title, cat], index=datasets.columns)
        datasets = datasets.append(s, ignore_index=True)

print(f"type: ${type(datasets)}")
print(f"shape: ${datasets.shape}")
datasets

type: $<class 'pandas.core.frame.DataFrame'>
shape: $(7376, 2)


,title,category
0,友人代表のスピーチ、独女はどうこなしている？\n,dokujo-tsushin
1,ネットで断ち切れない元カレとの縁\n,dokujo-tsushin
2,相次ぐ芸能人の“すっぴん”披露 その時、独女の心境は？\n,dokujo-tsushin
3,ムダな抵抗！？ 加齢の現実\n,dokujo-tsushin
4,税金を払うのは私たちなんですけど！\n,dokujo-tsushin
...,...,...
7371,爆笑問題・田中裕二も驚く「ひるおび!」での恵俊彰の“天然”ぶり\n,topic-news
7372,黒田勇樹のDV騒動 ネット掲示板では冷ややかな声も\n,topic-news
7373,サムスンのアンドロイド搭載カメラが韓国で話題に\n,topic-news
7374,米紙も注目したゲーム「竹島争奪戦」\n,topic-news


In [5]:
# データフレームシャッフル
datasets = datasets.sample(frac=1).reset_index(drop=True)
datasets.head()

,title,category
0,私たち、姪＆甥がかわいくて仕方ないんです！\n,dokujo-tsushin
1,逮捕されたソフトバンク・堂上隼人に非難が殺到\n,topic-news
2,【オトナ女子映画部】アラサーアイドルがヒーローに！安っぽさやくだらなさを楽しむ『エイトレンジ...,dokujo-tsushin
3,iPhoneユーザー歓喜！？コンビニ無線LAN「LAWSON Wi-Fi」がiPhoneに対応\n,smax
4,ワイヤレス・テクノジー・パーク2012：日本無線ブースにてスマートフォンをITS受信機にする...,smax


PyTorchでLSTMをする際、食わせるインプットデータは３次元のテンソルある必要があります。具体的には、文章の長さ × バッチサイズ × ベクトル次元数 となっています。今回のインプットデータは文章（livedoorニュースのタイトル文）であり、この文章を3次元テンソルに変換する必要があります。

バッチサイズは一旦無視して、ひとまず文章を以下のように２次元のマトリクスに変換することを考えます。

```
人口知能は人間の仕事を奪った
（形態素解析）→['人口','知能','は','人間','の','仕事','を','奪っ','た']
(各単語をベクトルで置換)→[[0.2 0.5 -0.9 1.3 ...], # 「人口」の単語ベクトル
                     [1.3 0.1 2.9 -1.3 ...], # 「知能」の単語ベクトル
                      ...
                     [0.9 -0.3 -0.1 3.0 ...] # 「た」の単語ベクトル
                    ]
```
単語のベクトルは例えばWord2Vecで学習済みのものがあればそれを使う方が精度が良いらしいですが、一旦はPyTorchの torch.nn.Embedding を使いましょう。こいつの詳細はPyTorchのチュートリアルに任せますが、要はランダムな単語ベクトル群を生成してくれるやつです。実際に使ってみると分かりやすいです。


In [6]:
import torch
torch.set_printoptions(linewidth=1000)

import torch.nn as nn
import MeCab
import re

# 実行する前に下記を実行すること
# cp -p /etc/mecabrc /usr//local/etc/

tagger = MeCab.Tagger("-Owakati")
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.device_count())

1.8.1+cu102
False
0


In [7]:
# 以下の宣言で行が単語ベクトル、列が単語のインデックスのマトリクスを生成してる感じ
embeds = nn.Embedding(10, 6) # (Embedding(単語の合計数（1つの文を構成する単語数）, ベクトル次元数))

In [8]:
print(f"type: ${type(embeds)}")

type: $<class 'torch.nn.modules.sparse.Embedding'>


In [9]:
embeds

Embedding(10, 6)

In [10]:
# ３行目の要素を取り出したいならば
w1 = torch.tensor([2])
print(embeds(w1))

tensor([[ 0.4335, -0.0524,  0.2694,  0.9246, -0.2736, -0.0444]], grad_fn=<EmbeddingBackward>)


In [11]:
# 3行目、5行目、１０行目の要素を取り出したいならば、
w2 = torch.tensor([2,4,9])
print(embeds(w2))

tensor([[ 0.4335, -0.0524,  0.2694,  0.9246, -0.2736, -0.0444],
        [ 0.3472, -1.1720, -0.9931,  0.2534,  0.5588,  1.7256],
        [-0.0286,  0.3725,  0.6421,  2.6326,  1.1246, -1.1369]], grad_fn=<EmbeddingBackward>)


torch.nn.Embeddingを使えば文章を簡単に２次元のマトリクスにすることができます。そのために、文章を単語IDの系列データとして変換すれば、さくっと文章を２次元のマトリクスにできそうです。文章を形態素解析して、全ての単語にIDを割り振って、文章を単語IDの系列データにする前の一連の流れは、例えば以下のような感じでよいでしょう。
（形態素解析にはとりえあずMeCab使います。前処理で英数字や記号は諸々削除していますが、実際は要件に応じて相談。）

In [12]:
def make_wakati(sentence):
    # MeCabで分かち書き
    sentence = tagger.parse(sentence)
    # 半角全角英数字除去
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    # スペースで区切って形態素の配列へ
    wakati = sentence.split(" ")
    # 空の要素は削除
    wakati = list(filter(("").__ne__, wakati))
    return wakati

In [13]:
# テスト
test = "【人工知能】は「人間」の仕事を奪った"
print(make_wakati(test))

['人工', '知能', 'は', '人間', 'の', '仕事', 'を', '奪っ', 'た']


In [14]:
# 単語ID辞書を作成する
word2index = {}
for title in datasets["title"]:
    wakati = make_wakati(title)
    for word in wakati:
        if word in word2index: continue
        word2index[word] = len(word2index)
print("vocab size : ", len(word2index))

vocab size :  13276


In [15]:
# 文章を単語IDの系列データに変換
# PyTorchのLSTMのインプットになるデータなので、もちろんtensor型で
def sentence2index(sentence):
    wakati = make_wakati(sentence)
    return torch.tensor([word2index[w] for w in wakati], dtype=torch.long)

# テスト
test = "例のあのメニューも！ニコニコ超会議のフードコートメニュー14種類紹介（前半）"
test_result = sentence2index(test)
print(test_result)
print(type(test_result))

tensor([8103,   64, 1047,  997,  134, 1602,  273, 1603,   64, 8144, 7069,  140, 8538])
<class 'torch.Tensor'>


In [16]:
# 単語のベクトル数
EMBEDDING_DIM = 10
# 全単語数を取得
VOCAB_SIZE = len(word2index)
VOCAB_SIZE

13276

In [17]:
test = "ユージの前に立ちはだかったJOY「僕はAKBの高橋みなみを守る」"
# 単語IDの系列データに変換
inputs = sentence2index(test)

print(type(inputs))
print(inputs)

<class 'torch.Tensor'>
tensor([9799,   64,  176,   18, 9800,   14,  519,   84,   64, 1381, 4248,   30, 6434])


In [18]:
# 各単語のベクトルをまとめて取得
# 以下の宣言で行が単語ベクトル、列が単語のインデックスのマトリクスを生成してる感じ
# (Embedding(単語の合計数（1つの文を構成する単語数）, ベクトル次元数))
embeds = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)

print(type(embeds))
print(embeds)

<class 'torch.nn.modules.sparse.Embedding'>
Embedding(13276, 10)


In [19]:
sentence_matrix = embeds(inputs)
print(type(sentence_matrix))
print(sentence_matrix.size())
print(sentence_matrix)
print("###################テスト終わり#####################")

<class 'torch.Tensor'>
torch.Size([13, 10])
tensor([[ 8.5704e-01, -1.1438e+00,  6.4939e-01, -1.0367e-01,  1.3721e+00, -4.1872e-01,  2.4071e+00, -1.3405e+00,  3.6437e-02,  1.6093e-01],
        [ 1.7554e-01,  2.9242e-01, -8.7265e-01, -5.2237e-02,  5.0813e-01,  2.0564e+00,  1.2138e-01, -1.8944e+00,  1.0731e-01, -4.3561e-01],
        [-2.8149e-02, -5.9173e-01, -4.0962e-01, -1.9348e+00,  3.1610e-01,  1.3705e+00, -1.8305e+00, -5.3600e-01, -1.5486e-01, -6.1525e-02],
        [ 4.8530e-01, -5.8736e-01,  4.9884e-01, -3.7233e-01,  6.3638e-01, -4.0580e-01, -2.5622e-01,  2.1412e+00, -4.9334e-01,  1.3339e+00],
        [ 5.0238e-02, -1.4846e+00,  1.9270e+00, -3.6990e-01, -5.7551e-01,  1.2508e+00, -9.2561e-01,  1.8859e-01, -2.9269e-01, -9.0330e-01],
        [-4.9547e-01, -5.1980e-01, -1.4891e+00,  6.4522e-01,  1.0830e+00, -1.6218e-01,  6.0020e-01, -2.4458e+00,  1.0878e+00,  1.2651e+00],
        [-1.3956e+00, -1.4083e+00,  3.3482e-01, -7.1853e-02,  9.3753e-01,  4.6177e-04,  6.3564e-01,  1.0854e+00,  3.

In [20]:
VOCAB_SIZE = len(word2index)
EMBEDDING_DIM = 10
HIDDEN_DIM = 128
embeds = nn.Embedding(VOCAB_SIZE, EMBEDDING_DIM)
s1 = "震災をうけて感じた、大切だと思ったこと"
print(make_wakati(s1))

['震災', 'を', 'うけ', 'て', '感じ', 'た', '大切', 'だ', 'と', '思っ', 'た', 'こと']


In [21]:
inputs1 = sentence2index(s1)
emb1 = embeds(inputs1)
lstm_inputs1 = emb1.view(len(inputs1), 1, -1)
lstm = nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM)
out1, out2 = lstm(lstm_inputs1)
print(f"out1.size():${out1.size()}")
print(out1)
print(f"out2[0].size():${out2[0].size()}")
print(f"out2[1].size():${out2[1].size()}")
print(out2)

out1.size():$torch.Size([12, 1, 128])
tensor([[[ 0.0061,  0.0085, -0.0231,  ..., -0.0142,  0.0287,  0.0123]],

        [[ 0.0234, -0.0810, -0.0107,  ..., -0.0354,  0.0503,  0.0090]],

        [[ 0.0766, -0.0149,  0.0247,  ...,  0.0161,  0.0862,  0.0104]],

        ...,

        [[-0.0695, -0.0210,  0.0856,  ...,  0.0198,  0.0383,  0.0714]],

        [[-0.0503,  0.0199,  0.0963,  ...,  0.0219,  0.0594,  0.0575]],

        [[-0.0248, -0.0254,  0.1292,  ...,  0.0692,  0.0794,  0.0824]]], grad_fn=<StackBackward>)
out2[0].size():$torch.Size([1, 1, 128])
out2[1].size():$torch.Size([1, 1, 128])
(tensor([[[-0.0248, -0.0254,  0.1292, -0.0355,  0.0104, -0.0645,  0.0366,  0.0763, -0.0858,  0.0273,  0.0443,  0.0019, -0.0011,  0.0123, -0.0121, -0.0128, -0.0241, -0.0420, -0.1082, -0.0537,  0.0611, -0.0095,  0.0192,  0.0554,  0.0166,  0.0227, -0.0064, -0.0253, -0.0063, -0.0208,  0.0339,  0.0085,  0.0835,  0.0932,  0.0089,  0.0070, -0.0098, -0.0683,  0.0022, -0.0649,  0.1160,  0.0326,  0.0378,  0.0999